# Group 5
## Members: Ruoyu Lin, Zhengyi Zhu, Ruopu Wang, Tiantian Zhang, Rain Wei

Goal: want to find best schedule $(n_0, n_1, ..., n_{T-1})$ that maximizes the total number of traded shares.

Moving average $M$ defined as 
$$M_{-1} = 0; M_t = \lceil 0.1M_{t-1}+0.9n_t \rceil, for\ 0\leq t\leq T-1$$

Impact as an extension of $M$:
$$impact_{t} = 1-\alpha M_t^\pi, for 0\leq t \leq T-1$$

Consequently, actually traded shares:
$$S_t = \lceil (1-\alpha M_t^\pi)n_t\rceil$$

The optimization problem can be characterized as:

$$
\max_{(n_0,n_1,...,n_{T-2},n_{T-1})} \lceil(1-\alpha M_0^\pi)n_0\rceil + \lceil(1-\alpha M_1^\pi)n_1\rceil +...+ \lceil(1-\alpha M_{T-1}^\pi)n_{T-1}\rceil \\
where\ M_t = \lceil 0.1M_{t-1}+0.9n_t \rceil
for\ 0\leq t\leq T-1; \\
where\ 0\leq n_t\leq N, \sum_{t=0}^{T-1}n_t=N 
$$

Let $total(t,n)$ be the max total traded shares we’ve made for the first $t+1$ time periods with the schedule $(n_0,n_1,...,n_{t})$ and $\sum_{j=0}^{t}n_j=n$. So clearly $\sum_{j=t+1}^{T-1}n_{j}=N-n$. Here, we have $0\leq t\leq T-1$ and $0\leq n\leq N$. 

Consider when $T=1$, we only have 1 period to execute trading, then we have to schedule all shares on this only period, namely $n_0$ is the total number of shares we want to arrange. 

So we have the initial boundary condition: $total(0,n) = \lceil (1-\alpha M_0^\pi)n\rceil = \lceil (1-\alpha \lceil 0.1*0+0.9n\rceil^\pi)n\rceil = \lceil1-\alpha (\lceil 0.9n\rceil^\pi)n\rceil$

Add one period, when $T=2$, we have 2 periods to execute trading, so we want to find $total(1,N)$ and corresponding $(n_0, n_1)$

$$
total(1,n)=\max_{0\leq h\leq n}[total(0,n-h)+\lceil (1-\alpha M_{1}^\pi) h\rceil]
$$

Add one period, when $T=3$, we have 3 periods to execute trading, so we want to find $total(2,N)$ and the corresponding $(n_0,n_1,n_2)$

$$
total(2,n)=\max_{0\leq h\leq n}[total(1,n-h)+\lceil (1-\alpha M_{2}^\pi) h\rceil]
$$

In general, for $1\leq t\leq T-1$,

\begin{align*}
    \color{green}{total(t,n)} &\color{green}{=\max_{0\leq h\leq n}[total(t-1,n-h)+\lceil (1-\alpha M_{t}^\pi)h\rceil]}\\
    &\color{green}{=\max_{0\leq h\leq n}[total(t-1,n-h)+\lceil (1-\alpha \lceil 0.1M_{t-1}+0.9h\rceil^\pi)h\rceil]}
\end{align*}

Notation: The following are 3 tables of size $(T,N+1)$:

- $total(t,n):=$ max traded shares for the first t+1 periods, optimal schedule $(n_0,n_1,...,n_t)$, with $\sum_{j=0}^{t}n_j=n$

- $S(t,n):=n_t$. That’s the optimal scheduled shares for the most recent period. 

- $M(t,n):=\lceil 0.1M(t-1,n-n_t)+0.9n_t\rceil$ . That’s the M value for the optimal schedule so far. 

which are initialized as follows:

- $total(0,n)=\lceil1-\alpha (\lceil 0.9n\rceil^\pi)n\rceil$

- $S(0,n)=n$

- $M(0,n)=\lceil 0.9n\rceil$

\begin{align*}
    \color{green}{total(t,n)} &\color{green}{=total(t-1,n-h)+\max_{0\leq h\leq n}[\lceil (1-\alpha M_{t}^\pi)h\rceil]}\\
    &\color{green}{=total(t-1,n-h)+\max_{0\leq h\leq n}[\lceil (1-\alpha \lceil 0.1M_{t-1}+0.9h\rceil^\pi)h\rceil]}
\end{align*}

# Task 1

Importation

In [1]:
import numpy as np
import pandas as pd
import datetime

from typing import Tuple

In [2]:
# Task 1
def extract_optimal_schedule(S: np.ndarray) -> np.ndarray:
    """
    Given the S matrix outputted by the DP algorithm, extract optimal schedule.


    Parameters
    --------------
    S: np.ndarray: The optimal S matrix where S[t,n] is the optimal sale volume
    when there are t periods left and n shares to sell.


    Returns
    --------------
    np.ndarray: optimal sale schedule stored in an numpy array.
    """
    # extract dimensions
    T,N = S.shape[0],S.shape[1]-1

    # initialize vector
    S_optimal = np.zeros(T)

    # initialize final period
    S_optimal[T-1] = S[T-1,N]

    # backfill
    for t in range(T-2,-1,-1):
        S_optimal[t] = S[t,int(N-sum(S_optimal[t+1:]))]
    return S_optimal

def dp(N:int, T:int, alpha:float, pi:float) -> np.ndarray:
    """
    The main DP algorithm for computing the optimal trading volume.


    Parameters
    --------------
    N: int: Total umber of shares to trade

    T: int: Total time periods available to trade

    alpha: float: trading execution decay parameter. Assumed to be small.

    pi: float: trading execution exponent parameter. Must be between 0 and 1.


    Returns
    --------------
    np.ndarray: optimal sale schedule stored in an numpy array.
    """
    # initializing total, S, M
    total = np.zeros((T,N+1))
    S = np.zeros((T,N+1))
    M = np.zeros((T,N+1))

    # helper vectors
    iarray = np.arange(N+1)
    revi = N-iarray
    candidates = np.zeros(N+1)

    # initialize total, S, M for t=0
    np.copyto(M[0,:], np.ceil(0.9*iarray))
    np.copyto(S[0,:], iarray)
    np.copyto(total[0,:], np.ceil((1-alpha*M[0,:]**pi)*iarray))

    # run DP algorithm
    for t in range(1,T):
        for n in range(N+1):
            np.copyto(candidates[:n+1], total[t-1,revi[N-n:]]+np.ceil((1-alpha*np.ceil(0.1*M[t-1,revi[N-n:]]+0.9*iarray[:n+1])**pi)*iarray[:n+1]))
            idx_max = np.argmax(candidates[:n+1])
            total[t,n] = candidates[:n+1][idx_max]
            S[t,n] = idx_max
            M[t,n] = np.ceil(0.1*M[t-1,int(n-S[t,n])]+0.9*S[t,n])

    # extract schedule
    S_optimal = extract_optimal_schedule(S)
    traded = total[T-1, N]

    # clear memory and return
    del total, M, S

    return S_optimal, traded

In [3]:
N, T, alpha, pi = 100000, 10, 0.001, 0.5
S_optimal, traded = dp(N, T, alpha, pi)
print(f"Parameters: N={N}, T={T}, alpha={alpha}, pi={pi}")
print("Optimal Schedule:")
print(S_optimal)
print("Total shares traded:")
print(int(traded))
print("Traded percentage of target:")
print(traded/N)

Parameters: N=100000, T=10, alpha=0.001, pi=0.5
Optimal Schedule:
[10100.  9900.  9840.  9888.  9976. 10001. 10048. 10067. 10087. 10093.]
Total shares traded:
90067
Traded percentage of target:
0.90067


# Task 2

In [4]:
# Task 2

def filter_first_2hours(dt: datetime.datetime) -> bool:
    """
    Task 2: Helper flag function to determine whether a datetime is within
    2 hours of market open or not.


    Parameters
    --------------
    dt: datetime.datetime: input datetime


    Returns
    --------------
    bool: is first 2 hours or not
    """
    return dt.time() >= datetime.time(9,30) and dt.time() < datetime.time(11, 30)

def ingest_tsla_volume(filename: str = "TSLA.csv") -> np.ndarray:
    """
    Task 2: ingest data.


    Parameters
    --------------
    filename: str: the filepath of the TSLA.csv file.


    Returns
    --------------
    np.ndarray: first half of all volume data 
    """

    # use pd.read_csv because need to use datetime
    df = pd.read_csv(filename, low_memory=False)
    # remove headers
    df = df.iloc[3:, :]
    # only read valid data
    numactual = df.iloc[:, 6].astype(float).isna().argmax()
    df = df.iloc[:numactual]
    # take half of all valid data, down to the last whole day
    l = len(df)//2
    l -= l % 10
    df = df.iloc[:l]
    # filter out time after 2hrs since market open
    dt = pd.to_datetime(df["BarTp"])
    filt_f2h = dt.apply(filter_first_2hours)
    volume = df.loc[filt_f2h].iloc[:, 6].values.astype(int)
    return volume.reshape(len(volume)//10, 10)


def evaluate_total_score(volume_all: np.ndarray, schedule: np.ndarray = None) -> float:
    """
    Task 2: evaluate trading schedule on all TSLA volume data.


    Parameters
    --------------
    volume_all: np.ndarray: all volume data needed.

    schedule: np.ndarray: optimal trading schedule as computed in Task 1, optional.


    Returns
    --------------
    float: total score
    """
    # extract dimensions
    L = volume_all.shape[0]
    assert volume_all.shape[1] == 10

    # to compute score via minimum
    SV = np.zeros((2, L, 10))

    # reshape input and copy
    np.copyto(SV[0, :, :], volume_all/100)
    np.copyto(SV[1, :, :], np.vstack([schedule for _ in range(L)]))

    return np.min(SV, axis=0).sum(axis=1).mean()

In [5]:
TSLA_volume = ingest_tsla_volume()
print("TSLA Volume (by interval):")
print(TSLA_volume)

TSLA Volume (by interval):
[[736210 189614 277161 ... 218539 192662 130332]
 [351514 263425 322540 ... 274423 179297 177969]
 [175444 149933 173485 ... 155897  80435 101384]
 ...
 [ 66514  63346  67210 ...  22548  20338  12258]
 [ 44428  42148  20466 ...  80808  25821  15837]
 [ 38977  39590  44390 ...  24904  27353  22822]]


In [6]:
print("Total score using optimal schedule obtained in Task 1:")
evaluate_total_score(volume_all=TSLA_volume, schedule=S_optimal)

Total score using optimal schedule obtained in Task 1:


8113.977045454546

# Task 3

To save time, we pre-ran optimal schedules for $\pi$ parameters grids of different frequency within $(0.3, 0.7)$ for different batch size $(10, 100)$ shares per batch to facilitate development and to accelerate computation. These files are stored in the same directory as this notebook, namely `optimalS_<batch_size>_<size_of_grid>.npy` for batch size $\times$ size of grid $=(10, 100)\times(41, 401)$. 

Each `.npy` file characterizes the optimal schedule for all $\pi$, and is of shape (number of $\pi$ values, 10) i.e. contains exactly <number of $\pi$ values> optimal schedules for the specific batch size in the file name.

In [7]:
def get_max_total_score(N,T,alpha,TSLA_volume, num_pi,batch):
    pi_choices = np.linspace(0.3,0.7,num_pi)

    S_record = np.zeros((len(pi_choices), 10))
    total_score_record = np.zeros(len(pi_choices))

    for i in range(len(pi_choices)):
        S, _= dp(N,T,alpha,pi_choices[i])
        np.copyto(S_record[i,:], S*batch)
        total_score_record[i] = evaluate_total_score(TSLA_volume, S_record[i,:])

    np.save(f'./S_record_{num_pi}.npy', S_record)
    np.save(f'./total_score_record_{num_pi}.npy', total_score_record)

    idx_max = np.argmax(total_score_record)
    return pi_choices[idx_max], total_score_record[idx_max], S_record[idx_max,:]

The function referenced above is used to pre-compute the optimal schedules for different parameter mentioned above. In particular, the `np.save` lines performed the caching operation. This function is simply the batch-ified version of the `evaluate_total_score` function above.

In [8]:
# Task 3

def get_max_total_score_from_npy(
        TSLA_volume: np.ndarray, 
        num_pi:int, batch:int) -> Tuple[float, float, np.ndarray]:
    """
    Task 3: tune parameter pi to obtain optimizer of max total score.


    Parameters
    --------------
    N: int: Total umber of shares to trade

    T: int: Total time periods available to trade

    alpha: float: trading execution decay parameter. Assumed to be small.

    TSLA_volume: np.ndarray: TSLA volume read from file

    num_pi: number of pi values between 0.3 and 0.7, linearly spanned

    batch: number of shares per batch.


    Returns
    --------------
    Tuple[float, float, np.ndarray]: optimal pi, optimal max total score,
    optimal schedule as obtained by the optimal pi.
    """
    # initialize grid for pi values
    pi_choices = np.linspace(0.3,0.7,num_pi)

    # initialize vectors
    total_score_record = np.zeros(len(pi_choices))
    filename="optimalS_"+str(batch)+"_"+str(num_pi)+".npy"

    # loop through different pi value
    S = np.load(filename)
    for i in range(len(pi_choices)):
        # compute total score, benchmarked on TSLA volume data
        total_score_record[i] = evaluate_total_score(volume_all=TSLA_volume, schedule = S[i,:])

    # get optimizer index
    idx_max = np.argmax(total_score_record)
    return pi_choices[idx_max], total_score_record[idx_max], S[idx_max]

In [9]:
# Ingest data using task 2 code
TSLA_volume = ingest_tsla_volume('TSLA.csv')

# Below lists are for reading static files
batch = [100,10]
num_pi = [401,41]

# Define params
alpha = 0.001
N = int(100000)
T = 10

# Compute score
for batch_num in batch:
    for pi in num_pi:
        pi_max, total_record_max, S_optimal = get_max_total_score_from_npy(TSLA_volume, pi,batch_num)
        print("*"*65)
        print(f"Batch size: {batch_num}; Number of pi: {pi}; Grid frequency: {round((0.7-0.3)/(pi-1), 3)}")
        print(f"Optimal pi = {pi_max}")
        print(f"Optimal schedule = {S_optimal}")
        print(f"Optimized Total Score = {total_record_max}")
        print("*"*65)
        print()

*****************************************************************
Batch size: 100; Number of pi: 401; Grid frequency: 0.001
Optimal pi = 0.525
Optimal schedule = [15100.  9100.  9200.  9200.  9200.  9200.  9200.  9200. 14700.  5900.]
Optimized Total Score = 8121.400770202021
*****************************************************************

*****************************************************************
Batch size: 100; Number of pi: 41; Grid frequency: 0.01
Optimal pi = 0.5399999999999999
Optimal schedule = [14400.  8700.  8800.  8800.  8800.  8800.  8800. 14000. 13900.  5000.]
Optimized Total Score = 8121.400770202021
*****************************************************************

*****************************************************************
Batch size: 10; Number of pi: 401; Grid frequency: 0.001
Optimal pi = 0.311
Optimal schedule = [11530. 10380. 10380. 10380. 10380. 10380. 10380. 10380. 10380.  5430.]
Optimized Total Score = 8118.5990656565655
***************************